In [ ]:
import os
import hashlib
from collections import Counter

import numpy as np
import matplotlib.pyplot as plt
from skimage.util import montage
import cv2
from cv2 import img_hash

from sdcdup.utils import overlap_tag_pairs
from sdcdup.utils import generate_overlap_tag_slices
from sdcdup.utils import boundingbox_corners
from sdcdup.utils import channel_shift
from sdcdup.utils import load_duplicate_truth

from test_friend_circles import SDCImageContainer

%matplotlib inline
%reload_ext autoreload
%autoreload 2

RED = (244, 67, 54) #F44336 
GREEN = (76, 175, 80) #4CAF50 
LIGHT_BLUE = (3, 169, 244) #03A9F4

SMALL_SIZE = 10
MEDIUM_SIZE = 12
BIGGER_SIZE = 16
BIGGEST_SIZE = 20
plt.rc('font', size=BIGGEST_SIZE)         # controls default text sizes
plt.rc('axes', titlesize=BIGGEST_SIZE)    # fontsize of the axes title
plt.rc('axes', labelsize=BIGGEST_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=BIGGER_SIZE)   # fontsize of the tick labels
plt.rc('ytick', labelsize=BIGGER_SIZE)   # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)   # legend fontsize
plt.rc('figure', titlesize=BIGGEST_SIZE)  # fontsize of the figure title

montage_rgb = lambda x: np.stack([montage(x[:, :, :, i]) for i in range(x.shape[3])], -1)
montage_pad = lambda x, *args, **kwargs: montage(x, padding_width=10, *args, **kwargs)
zeros_mask = np.zeros((256*3, 256*3, 1), dtype=np.float32)

ship_dir = "data/input"
train_image_dir = os.path.join(ship_dir, "train_768")
train_mask_dir = os.path.join(ship_dir, 'train_masks_768')
train_seg_file = os.path.join(ship_dir, "fullmasks_768.h5")
image_md5hash_grids_file = os.path.join("data", "image_md5hash_grids.pkl")
image_bm0hash_grids_file = os.path.join("data", "image_bm0hash_grids.pkl")
image_cm0hash_grids_file = os.path.join("data", "image_cm0hash_grids.pkl")
image_greycop_grids_file = os.path.join("data", "image_greycop_grids.pkl")
image_entropy_grids_file = os.path.join("data", "image_entropy_grids.pkl")
image_issolid_grids_file = os.path.join("data", "image_issolid_grids.pkl")
image_shipcnt_grids_file = os.path.join("data", "image_shipcnt_grids.pkl")

overlap_tag_slices = generate_overlap_tag_slices()

dtick = 256
n_ticks = 768 // dtick + 1
ticks = [i * dtick for i in range(n_ticks)]

In [ ]:
class ImgMod:
    """
    Reads a single image to be modified by hls.
    """

    def __init__(self, filename):
        self.filename = filename
        self.img_id = filename.split('/')[-1]

        self._hls_chan = None
        self._hls_gain = None

        self._parent_bgr = None
        self._parent_hls = None
        self._parent_rgb = None
        self._cv2_hls = None
        self._cv2_bgr = None
        self._cv2_rgb = None

    def channel_shift(self, chan, gain):
        self._hls_chan = chan
        self._hls_gain = gain
        self._cv2_hls = None
        return self.cv2_rgb
    
    def scale(self, minval, maxval):
        m = 255.0 * (maxval - minval)
        res = m * (self.parent_bgr - minval)
        return np.around(res).astype(np.uint8)
    
    @property
    def shape(self):
        return self.parent_bgr.shape
    
    @property
    def parent_bgr(self):
        if self._parent_bgr is None:
            self._parent_bgr = cv2.imread(self.filename)
        return self._parent_bgr

    @property
    def parent_hls(self):
        if self._parent_hls is None:
            self._parent_hls = self.to_hls(self.parent_bgr)
        return self._parent_hls

    @property
    def parent_rgb(self):
        if self._parent_rgb is None:
            self._parent_rgb = self.to_rgb(self.parent_bgr)
        return self._parent_rgb

    @property
    def cv2_hls(self):
        if self._cv2_hls is None:
            if self._hls_gain == None:
                self._cv2_hls = self.parent_hls
            else:
                self._cv2_hls = channel_shift(self.parent_hls, self._hls_chan, self._hls_gain)
        return self._cv2_hls

    @property
    def cv2_bgr(self):
        if self._cv2_bgr is None:
            self._cv2_bgr = self.to_bgr(self.cv2_hls)
        return self._cv2_bgr

    @property
    def cv2_rgb(self):
        if self._cv2_rgb is None:
            self._cv2_rgb = self.to_rgb(self.cv2_bgr)
        return self._cv2_rgb

    def to_hls(self, bgr):
        return cv2.cvtColor(bgr, cv2.COLOR_BGR2HLS_FULL)

    def to_bgr(self, hls):
        return cv2.cvtColor(hls, cv2.COLOR_HLS2BGR_FULL)

    def to_rgb(self, bgr):
        return cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)

In [ ]:
sdcic = SDCImageContainer(train_image_dir)
sdcic.preprocess_image_properties(
    image_md5hash_grids_file,
    image_bm0hash_grids_file,
    image_cm0hash_grids_file,
    image_greycop_grids_file,
    image_entropy_grids_file,
    image_issolid_grids_file)
sdcic.preprecess_ship_counts(train_seg_file, image_shipcnt_grids_file)

In [ ]:
dup_truth = load_duplicate_truth()
print(len(dup_truth))

## Check to see how many exact duplicate tiles we have.

In [ ]:
dup_tiles = []
dup_hashes = {}
dup_files = []
dup_counts = {}
for img_id, tile_hashes in sdcic.tile_md5hash_grids.items():
    img = None
    c0 = Counter(tile_hashes)
    for i, c in c0.items():
        if c == 1:
            continue
        for ii in np.where(tile_hashes == i)[0]:
            tile_hash = sdcic.tile_md5hash_grids[img_id][ii]
            if tile_hash in dup_hashes:
                dup_counts[tile_hash] += 1
                if img_id not in dup_hashes[tile_hash]:
                    dup_hashes[tile_hash][img_id] = []
                dup_hashes[tile_hash][img_id].append(ii)
            else:
                dup_counts[tile_hash] = 1
                dup_hashes[tile_hash] = {}
                dup_hashes[tile_hash][img_id] = []
                dup_hashes[tile_hash][img_id].append(ii)
                dup_files.append(img_id)
                img = sdcic.get_img(img_id)
                new_tile = sdcic.get_tile(img, ii)
                dup_tiles.append(new_tile)
                print(len(dup_files) - 1, tile_hash, img_id, ii)
dup_counts

In [ ]:
for ii, dup_tile in enumerate(dup_tiles):
    print(ii)
    print(dup_tile[2, 2], dup_tile[2, -2])
    print(dup_tile[-2, 2], dup_tile[-2, -2])

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 8))
img1 = cv2.cvtColor(cv2.imread(os.path.join(train_image_dir, dup_files[0])), cv2.COLOR_BGR2RGB)
ax1.set_xticks(ticks)
ax1.set_yticks(ticks)
ax1.imshow(img1)
img2 = cv2.cvtColor(cv2.imread(os.path.join(train_image_dir, dup_files[5])), cv2.COLOR_BGR2RGB)
ax2.set_xticks(ticks)
ax2.set_yticks(ticks)
ax2.imshow(img2);

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 8))
img1 = cv2.cvtColor(cv2.imread(os.path.join(train_image_dir, dup_files[1])), cv2.COLOR_BGR2RGB)
ax1.set_xticks(ticks)
ax1.set_yticks(ticks)
ax1.imshow(img1)
img2 = cv2.cvtColor(cv2.imread(os.path.join(train_image_dir, dup_files[2])), cv2.COLOR_BGR2RGB)
ax2.set_xticks(ticks)
ax2.set_yticks(ticks)
ax2.imshow(img2);

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 8))
img1 = cv2.cvtColor(cv2.imread(os.path.join(train_image_dir, dup_files[3])), cv2.COLOR_BGR2RGB)
ax1.set_xticks(ticks)
ax1.set_yticks(ticks)
ax1.imshow(img1)
img2 = cv2.cvtColor(cv2.imread(os.path.join(train_image_dir, dup_files[4])), cv2.COLOR_BGR2RGB)
ax2.set_xticks(ticks)
ax2.set_yticks(ticks)
ax2.imshow(img2);

In [ ]:
black_tile = np.zeros((256, 256, 3), dtype=np.uint8)
white_tile = black_tile + 255
blue_tile = np.copy(black_tile)
blue_tile[:, :, 0] = 255
red_tile = np.copy(black_tile)
red_tile[:, :, 2] = 255
color_tiles = [black_tile, white_tile, blue_tile, red_tile]
for color_tile in color_tiles:
    print(img_hash.blockMeanHash(color_tile, mode=0)[0])
    print(hashlib.md5(color_tile.tobytes()).hexdigest())

In [ ]:
black_images = ['03ffa7680.jpg', '8d5521663.jpg', '5a70ef013.jpg', '9a2f9d347.jpg', '37a912dca.jpg', '4add7aa1d.jpg', '3db3ef7cc.jpg', '73fec0637.jpg', '7df214d98.jpg', 'c2955cd21.jpg', 'de018b2a8.jpg', '8ce769141.jpg', 'fc0e22a0a.jpg', '770c46cd4.jpg', 'd6e432b79.jpg', 'd5d1b6fb8.jpg', '0e4d7dd93.jpg', '9ddeed533.jpg', 'addc11de0.jpg', '65418dfe4.jpg', '119d6a3d6.jpg', '1b287c905.jpg', 'b264b0f96.jpg', '996f92939.jpg', 'e5c3b1f59.jpg']
fig, ax = plt.subplots(5, 5, figsize=(15, 15))
for i, img_id in enumerate(black_images):
    img = cv2.cvtColor(cv2.imread(os.path.join(train_image_dir, img_id)), cv2.COLOR_BGR2RGB)
    ax[i // 5, i % 5].imshow(img)
    ax[i // 5, i % 5].set_title(img_id)
    ax[i // 5, i % 5].set_xticks(ticks)
    ax[i // 5, i % 5].set_yticks(ticks)

plt.tight_layout()

In [ ]:
bbox_color_map = {
    'red': RED,
    'blue': LIGHT_BLUE,
    'green': GREEN
}
def draw_image_pair(img1_id, img2_id, img1_overlap_tag=None, shift_brightness=False, bbox_color=None, save=False):
    
    img1_overlap_tag = img1_overlap_tag or "08"

#     scores = overlap_image_maps[(img1_id, img2_id)][img1_overlap_tag]
    imgmod1 = ImgMod(os.path.join(train_image_dir, img1_id))
    imgmod2 = ImgMod(os.path.join(train_image_dir, img2_id))
    
    dtick = 256
    n_ticks = imgmod1.shape[1] // dtick + 1
    ticks = [i * dtick for i in range(n_ticks)]

    slice1 = overlap_tag_slices[img1_overlap_tag]
    slice2 = overlap_tag_slices[overlap_tag_pairs[img1_overlap_tag]]

    m12 = np.median(np.vstack([imgmod1.parent_rgb[slice1], imgmod2.parent_rgb[slice2]]), axis=(0, 1), keepdims=True).astype(np.uint8)
    img1_drop = imgmod1.parent_rgb - m12
    img2_drop = imgmod2.parent_rgb - m12
    
    if shift_brightness:
        brightness_level = -100 if np.sum(m12) >= 384 else 100
        img1 = imgmod1.channel_shift('L', brightness_level)
        img2 = imgmod2.channel_shift('L', brightness_level)
    else:
        img1 = imgmod1.parent_rgb
        img2 = imgmod2.parent_rgb
    
    if bbox_color:
        bbox_color = bbox_color_map[bbox_color]
    else:
        bbox_color = LIGHT_BLUE
        if (img1_id, img2_id, img1_overlap_tag) in dup_truth:
            bbox_color = GREEN if dup_truth[(img1_id, img2_id, img1_overlap_tag)] else RED
        
    bbox_thickness = 4
    offset = (bbox_thickness // 2) + 1
    offset_array = np.array([[offset], [-offset]])
    img1_bbox_pt1, img1_bbox_pt2 = boundingbox_corners[img1_overlap_tag] + offset_array
    img2_bbox_pt1, img2_bbox_pt2 = boundingbox_corners[overlap_tag_pairs[img1_overlap_tag]] + offset_array
    
    img1[slice1] = imgmod1.parent_rgb[slice1]
    img2[slice2] = imgmod2.parent_rgb[slice2]
    cv2.rectangle(img1, tuple(img1_bbox_pt1), tuple(img1_bbox_pt2), bbox_color, bbox_thickness)
    cv2.rectangle(img2, tuple(img2_bbox_pt1), tuple(img2_bbox_pt2), bbox_color, bbox_thickness)

    fig, ax = plt.subplots(2, 2, figsize=(15, 15))
    ax[0][0].imshow(img1)
    ax[0][0].set_title(f'{img1_id}')
    ax[0][0].set_xticks(ticks)
    ax[0][0].set_yticks(ticks)

    ax[0][1].imshow(img2)
    ax[0][1].set_title(f'{img2_id}')
    ax[0][1].set_xticks(ticks)
    ax[0][1].set_yticks(ticks)
    
    img1[slice1] = img1_drop[slice1]
    img2[slice2] = img2_drop[slice2]
    cv2.rectangle(img1, tuple(img1_bbox_pt1), tuple(img1_bbox_pt2), bbox_color, bbox_thickness)
    cv2.rectangle(img2, tuple(img2_bbox_pt1), tuple(img2_bbox_pt2), bbox_color, bbox_thickness)

    ax[1][0].imshow(img1)
#     ax[1][0].set_title(f'cor: {np.min(scores.cor):7.5f} {np.max(scores.cor):7.5f}')
    ax[1][0].set_xticks(ticks)
    ax[1][0].set_yticks(ticks)

    ax[1][1].imshow(img2)
#     ax[1][1].set_title(f'enp: {np.min(scores.enp):5.3f} {np.max(scores.enp):5.3f} {max(scores.pix)}')
    ax[1][1].set_xticks(ticks)
    ax[1][1].set_yticks(ticks)
    
    if save:
        filename = os.path.join('temp', f"{img1_id}_{img2_id}_{img1_overlap_tag}.jpg")
        if os.path.exists(filename):
            print(f"{filename} already exists.")
        else:
            fig.savefig(filename)
            print(f"{filename} saved.")


In [ ]:
img1_id = "46b87e21c.jpg"
img2_id = "f881c203f.jpg"
draw_image_pair(img1_id, img2_id, img1_overlap_tag="04", bbox_color='red')

In [ ]:
draw_image_pair(img1_id, img2_id, img1_overlap_tag="07")

In [ ]:
img1_id = "9b34f2f64.jpg"
img2_id = "e8b058856.jpg"
draw_image_pair(img1_id, img2_id, img1_overlap_tag="05", bbox_color='red')

In [ ]:
draw_image_pair(img1_id, img2_id, img1_overlap_tag="18")

In [ ]:
img1_id = "356f4c539.jpg"
img2_id = "6dd7430f6.jpg"
draw_image_pair(img1_id, img2_id, img1_overlap_tag="02")